In [10]:
import pathlib
import os
if not "changed_working_directory" in locals():
    changed_working_directory = True
    os.chdir(pathlib.Path().resolve().parent)
    
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import torch
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from utils.tictoc import tic, toc

from src import constants as const

if const.dir.project() != os.getcwd():
    raise ValueError("Project directory not the same as in consts")

In [12]:
dataset_train_path = os.path.join(const.dir.data(), "df_adult_edited_train.csv")
dataset_test_path = os.path.join(const.dir.data(), "df_adult_edited_test.csv")

data_train = pd.read_csv(dataset_train_path)
data_test = pd.read_csv(dataset_test_path)
discrete_columns = data_train.columns[data_train.dtypes == "object"]

In [13]:
from sdv.tabular import CTGAN

In [14]:
?CTGAN

In [8]:
# tic()
# model = CTGAN(epochs=100, discriminator_steps=10, verbose=1, cuda=False)
# model.fit(data_train)
# toc()
print("Elapsed time: 2971.843488 seconds.")

Elapsed time: 2971.843488 seconds.


In [23]:
n_synthetic_datasets=10
overwrite=False
dir_gen_data_ctgan = os.path.join(const.dir.data_gen(), "CTGAN")
from IPython.display import clear_output

with tqdm(total=n_synthetic_datasets) as pbar:
    for i in range(n_synthetic_datasets):
        curr_path =  os.path.join(dir_gen_data_ctgan, f"gen{i}.csv")
        if overwrite or not os.path.exists(curr_path):
            model = CTGAN(epochs=100, batch_size=500, discriminator_steps=10, verbose=0, cuda=True,
                         embedding_dim=128, generator_dim=(256, 256), discriminator_dim=(256, 256),
                          pac=10, log_frequency=True
                         )
            model.fit(data_train)
            data = model.sample()
            data.to_csv(curr_path, index=False)
            clear_output()
        pbar.update(1)
        print(pbar)

100%|██████████| 10/10 [3:27:37<00:00, 1249.26s/it]


In [ ]:
dat = model.sample()

In [ ]:
dat.to_csv("temp_synthetic_data_cgan.csv", index=False)

In [9]:
?evaluate

In [8]:
from sdv.evaluation import evaluate
#evaluate(dat, data_train)

In [1]:
import ctgan